In [ ]:
import os
os.chdir('..')

In [1]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [2]:
dataset_path = "./datasets/sell-in.txt"
productos_a_predecir = "./datasets/product_id_apredecir201912.txt"

In [3]:
df = pd.read_csv(dataset_path, sep='\t')

In [4]:
df['product_id'] = df['product_id'].astype(str)
df['periodo'] = df['periodo'].astype(str)

In [5]:
#df = df[~df['periodo'].isin(['201912', '201911'])]

In [6]:
df_a_predecir = pd.read_csv(productos_a_predecir, dtype={ 'product_id': str })

In [7]:
df['timestamp'] = pd.to_datetime(df['periodo'], format='%Y%m')

In [8]:
df = df[df['product_id'].isin(df_a_predecir['product_id'].unique())]

In [9]:
df_monthly_product = df.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [10]:
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [11]:
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [12]:
ts_data = ts_data.sort_index()

In [13]:
ts_data = ts_data.fill_missing_values()

In [14]:
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'
)

In [15]:
predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Users\dylan\Desktop\labo3-2025v\AutogluonModels\ag-20250720_143658'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.21
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
GPU Count:          1
Memory Avail:       20.41 GB / 27.66 GB (73.8%)
Disk Space Avail:   228.56 GB / 699.94 GB (32.7%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train_dat

In [16]:
forecast = predictor.predict(ts_data)

data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [17]:
forecast

mean         0.1          0.2          0.3  \
item_id timestamp                                                       
20001   2020-01-01  1299.947485  947.050106  1061.006683  1165.910008   
        2020-02-01  1269.824797  891.477226  1032.026034  1137.003878   
20002   2020-01-01  1149.595874  792.300940   913.178225  1029.177137   
        2020-02-01  1067.967993  721.338708   856.025286   938.583984   
20003   2020-01-01   790.374349  590.688798   657.256537   704.826174   
...                         ...         ...          ...          ...   
21266   2020-02-01     0.047042   -0.016368     0.005469     0.020314   
21267   2020-01-01     0.049668    0.013568     0.026739     0.035598   
        2020-02-01     0.049204    0.007889     0.022263     0.032051   
21276   2020-01-01     0.025028    0.009542     0.015189     0.018967   
        2020-02-01     0.025550    0.007465     0.014008     0.018355   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
20001   2020-01-01  1231.311688  1301.884423  1374.383249  1451.046521   
        2020-02-01  1201.699062  1266.409203  1355.029909  1428.010196   
20002   2020-01-01  1093.331678  1154.799315  1214.033834  1311.629051   
        2020-02-01  1003.819880  1064.600485  1128.727063  1226.438166   
20003   2020-01-01   753.356843   787.499410   825.216066   869.109212   
...                         ...          ...          ...          ...   
21266   2020-02-01     0.033901     0.046975     0.060710     0.076131   
21267   2020-01-01     0.042460     0.049702     0.056726     0.065596   
        2020-02-01     0.040406     0.049321     0.057938     0.068004   
21276   2020-01-01     0.021975     0.025080     0.028102     0.031859   
        2020-02-01     0.021878     0.025641     0.029286     0.033584   

                            0.8          0.9  
item_id timestamp                             
20001   2020-01-01  1532.734047  1673.073043  
        2020-02-01  1516.367107  1657.914809  
20002   2020-01-01  1393.653128  1511.659540  
        2020-02-01  1306.474220  1437.348906  
20003   2020-01-01   923.323578  1014.172175  
...                         ...          ...  
21266   2020-02-01     0.094050     0.125738  
21267   2020-01-01     0.076069     0.093455  
        2020-02-01     0.079650     0.100289  
21276   2020-01-01     0.036305     0.043448  
        2020-02-01     0.038666     0.047443  

[1560 rows x 10 columns]

In [18]:
forecast_mean = forecast['mean'].reset_index()
forecast_mean.columns

Index(['item_id', 'timestamp', 'mean'], dtype='object')

In [19]:
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

In [20]:
resultado = resultado[['item_id', 'timestamp', 'mean']]
resultado.columns = ['product_id', 'periodo', 'tn']
resultado

,product_id,periodo,tn
1,20001,2020-02-01,1269.824797
3,20002,2020-02-01,1067.967993
5,20003,2020-02-01,673.230149
7,20004,2020-02-01,497.243019
9,20005,2020-02-01,486.099335
...,...,...,...
1551,21263,2020-02-01,0.024197
1553,21265,2020-02-01,0.045126
1555,21266,2020-02-01,0.047042
1557,21267,2020-02-01,0.049204


In [21]:
resultado.to_csv("forecast_autogluon_ene_feb.csv", index=False)
resultado.head()

,product_id,periodo,tn
1,20001,2020-02-01,1269.824797
3,20002,2020-02-01,1067.967993
5,20003,2020-02-01,673.230149
7,20004,2020-02-01,497.243019
9,20005,2020-02-01,486.099335
